In [50]:
import pandas as pd
import numpy as np
import google_analytics_reporting_api as ga
import json as json
import pandas as pd
import numpy as np
import datetime
import googlemaps
import glob
# google_analytics_reporting_api functions writen by Jay in the path '/home/jian/Projects/Saatva/GA/google_analytics_reporting_api.py'

In [51]:
Zip_Deduplated=pd.read_excel("/home/jian/Projects/Saatva/Otherinput/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
Zip_Deduplated=Zip_Deduplated.iloc[:,[0,2,6]]
Zip_Deduplated.columns=['zipcode','dma','ST']
Zip_Deduplated=Zip_Deduplated.drop_duplicates(['zipcode'])
Zip_Deduplated.reset_index(inplace=True)
del Zip_Deduplated['index']
Zip_Deduplated=Zip_Deduplated[Zip_Deduplated['ST']!="HI"]
Zip_Deduplated=Zip_Deduplated[Zip_Deduplated['ST']!="AK"]

In [52]:
Sales_2018=pd.read_csv("/home/jian/Projects/Saatva/Data/transactions_for_jubaplus-20180328.csv",dtype=str)
Sales_2018=Sales_2018[~pd.isnull(Sales_2018['Order ID'])]
Sales_2018.reset_index(inplace=True)
del Sales_2018['index']
Sales_2018['Date']=Sales_2018['Date'].apply(lambda x: datetime.datetime.strptime(x,"%d-%b-%Y").date())
Sales_2018=Sales_2018[Sales_2018['Date']>=datetime.datetime.strptime("2018-02-01","%Y-%m-%d").date()]
Sales_2018=Sales_2018[Sales_2018['Date']<=datetime.datetime.strptime("2018-03-27","%Y-%m-%d").date()]

In [53]:
Sales=Sales_2018.copy()

In [54]:
# Clean zips in the 50 states of USA
Sales['Shipping Zip']=Sales['Shipping Zip'].astype(str).apply(lambda x:x.split("-")[0])
Sales['Shipping Zip']=Sales['Shipping Zip'].apply(lambda x: np.where(len(x)<5,x.zfill(5),x))
Sales=Sales[Sales['Shipping Zip'].apply(lambda x: len(x))==5]

Sales['Product Revenue']=Sales['Product Revenue'].astype(float)
Sales['zip_num_index']=pd.to_numeric(Sales['Shipping Zip'], errors='coerce')
Sales=Sales[~pd.isnull(Sales['zip_num_index'])]
Sales.reset_index(inplace=True)
del Sales['zip_num_index']
del Sales['index']
Sales=Sales.rename(columns={"Shipping Zip":"zipcode"})
Sales_by_Zip_Long=pd.merge(Sales,Zip_Deduplated,on="zipcode",how="left")
Sales_by_Zip_Long=Sales_by_Zip_Long[~pd.isnull(Sales_by_Zip_Long['ST'])]
Sales_by_Zip_Long.reset_index(inplace=True)
del Sales_by_Zip_Long['index']

In [55]:
Sales_by_Zip_Long.to_csv("/home/jian/Projects/Saatva/output/sales_us_day_after_president_day.csv")